# Create Custom Environtment

In [7]:
import os

dependencies_dir = "./Dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

In [9]:
%%writefile {dependencies_dir}/conda.yaml
name: lstm-ae-env
channels:
  - conda-forge
  - defaults
dependencies:
  - python=3.8
  - numpy>=1.21.2
  - pandas>=1.1,<1.2
  - pip
  - pip:
    - mlflow==2.4.1
    - azureml-mlflow==1.51.0
    - torch==1.12.0
    - torchvision==0.13.0
    - torchaudio==0.12.0
    - scikit-learn==0.24.2
    - matplotlib>=3.3.0
    - seaborn>=0.11.0
    - tqdm>=4.59.0
    - pytorch-lightning>=1.5.0  # Opsional untuk pelatihan lebih terstruktur
    - joblib>=1.0.0             # Untuk menyimpan model
    - jsonschema>=4.0.0         # Untuk validasi format data input/output


Writing ./Dependencies/conda.yaml


In [16]:
import os
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Environment
from azure.identity import DefaultAzureCredential

# Pastikan direktori yang berisi conda.yaml
dependencies_dir = "./Dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

# Simpan file conda.yaml ke lokasi dependencies_dir
conda_file_path = os.path.join(dependencies_dir, "conda.yaml")
with open(conda_file_path, "w") as f:
    f.write("""
name: lstm-ae-env
channels:
  - conda-forge
  - defaults
dependencies:
  - python=3.8
  - numpy>=1.21.2
  - pandas>=1.1,<1.2
  - pip
  - pip:
    - mlflow==2.4.1
    - azureml-mlflow==1.51.0
    - torch==1.12.0
    - torchvision==0.13.0
    - torchaudio==0.12.0
    - scikit-learn==0.24.2
    - matplotlib>=3.3.0
    - seaborn>=0.11.0
    - tqdm>=4.59.0
    - pytorch-lightning>=1.5.0
    - joblib>=1.0.0
    - jsonschema>=4.0.0
""")

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="545de186-b8b5-4f23-aba9-b7f0a043666d",
    resource_group_name="ASDP",
    workspace_name="asdp-project",
)

# Definisikan custom environment
custom_env_name = "aml-pytorch-mlflow"

pipeline_job_env = Environment(
    name="aml-pytorch-mlflow",
    description="Custom environment for PyTorch and MLflow",
    tags={"framework": "PyTorch", "mlflow": "enabled"},
    conda_file=conda_file_path,
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    version="0.3.0",  # Gunakan versi baru
)

# Buat atau perbarui environment di Azure ML
pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)

print(
    f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
)


Environment with name aml-pytorch-mlflow is registered to workspace, the environment version is 0.3.0


# Built Component

## 1. One Features (OC)

### a. Data Prepartaion Components

In [1]:
from azure.ai.ml import MLClient, load_component
from azure.identity import DefaultAzureCredential
import os

# Path ke file predict.yml
prep_svm_component_path = "./Components/1. Data Preparation/oc_svm_data_prep.yml"

# Pastikan file YAML ada
if not os.path.exists(prep_svm_component_path):
    raise FileNotFoundError(f"File oc_lstm_data_prep.yml tidak ditemukan di {prep_svm_component_path}")

credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="545de186-b8b5-4f23-aba9-b7f0a043666d",
    resource_group_name="ASDP",
    workspace_name="asdp-project",
)

# Memuat komponen dari file YAML
print(f"Membaca komponen dari {prep_svm_component_path}...")
prep_svm_component = load_component(source=prep_svm_component_path)

# Mendaftarkan komponen ke Azure ML
print("Mendaftarkan komponen ke Azure ML Workspace...")
registered_component = ml_client.components.create_or_update(prep_svm_component)

print(f"Komponen {registered_component.name} versi {registered_component.version} berhasil didaftarkan.")


Membaca komponen dari ./Components/1. Data Preparation/oc_svm_data_prep.yml...
Mendaftarkan komponen ke Azure ML Workspace...


Uploading 1. Data Preparation (0.03 MBs): 100%|██████████| 25236/25236 [00:00<00:00, 289472.77it/s]




Komponen oc_svm_data_preparation versi 1.8 berhasil didaftarkan.


## data prep all (train test)

In [14]:
from azure.ai.ml import MLClient, load_component
from azure.identity import DefaultAzureCredential
import os

# Path ke file predict.yml
prep_test_component_path = "./Components/1. Data Preparation/oc_svm_data_prep_test.yml"

# Pastikan file YAML ada
if not os.path.exists(prep_test_component_path):
    raise FileNotFoundError(f"File oc_lstm_data_prep.yml tidak ditemukan di {prep_test_component_path}")

credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="545de186-b8b5-4f23-aba9-b7f0a043666d",
    resource_group_name="ASDP",
    workspace_name="asdp-project",
)

# Memuat komponen dari file YAML
print(f"Membaca komponen dari {prep_test_component_path}...")
prep_test_svm_component = load_component(source=prep_test_component_path)

# Mendaftarkan komponen ke Azure ML
print("Mendaftarkan komponen ke Azure ML Workspace...")
registered_component = ml_client.components.create_or_update(prep_test_svm_component)

print(f"Komponen {registered_component.name} versi {registered_component.version} berhasil didaftarkan.")


Membaca komponen dari ./Components/1. Data Preparation/oc_svm_data_prep_test.yml...
Mendaftarkan komponen ke Azure ML Workspace...


Uploading 1. Data Preparation (0.02 MBs): 100%|██████████| 22984/22984 [00:00<00:00, 219515.26it/s]




Komponen data_prep_test versi 1.2 berhasil didaftarkan.


In [13]:
from azure.ai.ml import MLClient, load_component
from azure.identity import DefaultAzureCredential
import os

# Path ke file predict.yml
prep_train_component_path = "./Components/1. Data Preparation/oc_svm_data_prep_train.yml"

# Pastikan file YAML ada
if not os.path.exists(prep_train_component_path):
    raise FileNotFoundError(f"File oc_lstm_data_prep.yml tidak ditemukan di {prep_train_component_path}")

# Memuat komponen dari file YAML
print(f"Membaca komponen dari {prep_train_component_path}...")
prep_train_svm_component = load_component(source=prep_train_component_path)

# Mendaftarkan komponen ke Azure ML
print("Mendaftarkan komponen ke Azure ML Workspace...")
registered_component = ml_client.components.create_or_update(prep_train_svm_component)

print(f"Komponen {registered_component.name} versi {registered_component.version} berhasil didaftarkan.")


Membaca komponen dari ./Components/1. Data Preparation/oc_svm_data_prep_train.yml...
Mendaftarkan komponen ke Azure ML Workspace...


Uploading 1. Data Preparation (0.02 MBs): 100%|██████████| 22778/22778 [00:00<00:00, 196800.24it/s]




Komponen data_prep_x_train versi 1.3 berhasil didaftarkan.


### b. Training Model

In [4]:
from azure.ai.ml import MLClient, load_component
from azure.identity import DefaultAzureCredential
import os

# Path ke file predict.yml
train_svm_component_path = "./Components/2. Training dan Predict Model/oc_one_class_svm_training_model.yml"

# Pastikan file YAML ada
if not os.path.exists(train_svm_component_path):
    raise FileNotFoundError(f"File oc_lstm_training_model.yml tidak ditemukan di {train_svm_component_path}")

# Memuat komponen dari file YAML
print(f"Membaca komponen dari {train_svm_component_path}...")
train_svm_component = load_component(source=train_svm_component_path)

# Mendaftarkan komponen ke Azure ML
print("Mendaftarkan komponen ke Azure ML Workspace...")
registered_component = ml_client.components.create_or_update(train_svm_component)

print(f"Komponen {registered_component.name} versi {registered_component.version} berhasil didaftarkan.")


Membaca komponen dari ./Components/2. Training dan Predict Model/oc_one_class_svm_training_model.yml...
Mendaftarkan komponen ke Azure ML Workspace...


Uploading 2. Training dan Predict Model (0.01 MBs): 100%|██████████| 10502/10502 [00:00<00:00, 302274.03it/s]




Komponen oc_svm_model_training versi 1.6 berhasil didaftarkan.


# semua data dijadikan train

In [1]:
from azure.ai.ml import MLClient, load_component
from azure.identity import DefaultAzureCredential
import os

# Path ke file predict.yml
train_all_svm_component_path = "./Components/2. Training dan Predict Model/oc_one_class_svm_training_model-v2.yml"

# Pastikan file YAML ada
if not os.path.exists(train_all_svm_component_path):
    raise FileNotFoundError(f"File oc_lstm_training_model.yml tidak ditemukan di {train_all_svm_component_path}")

credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="545de186-b8b5-4f23-aba9-b7f0a043666d",
    resource_group_name="ASDP",
    workspace_name="asdp-project",
)

# Memuat komponen dari file YAML
print(f"Membaca komponen dari {train_all_svm_component_path}...")
train_all_svm_component = load_component(source=train_all_svm_component_path)

# Mendaftarkan komponen ke Azure ML
print("Mendaftarkan komponen ke Azure ML Workspace...")
registered_component = ml_client.components.create_or_update(train_all_svm_component)

print(f"Komponen {registered_component.name} versi {registered_component.version} berhasil didaftarkan.")


Membaca komponen dari ./Components/2. Training dan Predict Model/oc_one_class_svm_training_model-v2.yml...
Mendaftarkan komponen ke Azure ML Workspace...


Uploading 2. Training dan Predict Model (0.02 MBs): 100%|██████████| 17326/17326 [00:00<00:00, 205864.30it/s]




Komponen oc_svm_train_model_v2 versi 1.0 berhasil didaftarkan.


### c. Evaluate Model

In [6]:
from azure.ai.ml import MLClient, load_component
from azure.identity import DefaultAzureCredential
import os

# Path ke file predict.yml
eval_svm_component_path = "./Components/3. Evaluate Model/oc_svm_evaluate_model.yml"

# Pastikan file YAML ada
if not os.path.exists(eval_svm_component_path):
    raise FileNotFoundError(f"File oc_lstm_data_prep.yml tidak ditemukan di {eval_svm_component_path}")

credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="545de186-b8b5-4f23-aba9-b7f0a043666d",
    resource_group_name="ASDP",
    workspace_name="asdp-project",
)

# Memuat komponen dari file YAML
print(f"Membaca komponen dari {eval_svm_component_path}...")
eval_svm_component = load_component(source=eval_svm_component_path)

# Mendaftarkan komponen ke Azure ML
print("Mendaftarkan komponen ke Azure ML Workspace...")
registered_component = ml_client.components.create_or_update(eval_svm_component)

print(f"Komponen {registered_component.name} versi {registered_component.version} berhasil didaftarkan.")


Membaca komponen dari ./Components/3. Evaluate Model/oc_svm_evaluate_model.yml...
Mendaftarkan komponen ke Azure ML Workspace...


Uploading 3. Evaluate Model (0.01 MBs): 100%|██████████| 10434/10434 [00:00<00:00, 101896.60it/s]




Komponen oc_svm_evaluate_model versi 1.6 berhasil didaftarkan.


## convert to cosmoss

In [1]:
from azure.ai.ml import MLClient, load_component
from azure.identity import DefaultAzureCredential
import os

# Path ke file predict.yml
input_cosmoss_component_path = "./Components/4. Integrate to Cosmoss DB/CSV_to_cosmos_DB.yml"

# Pastikan file YAML ada
if not os.path.exists(input_cosmoss_component_path):
    raise FileNotFoundError(f"File oc_lstm_data_prep.yml tidak ditemukan di {input_cosmoss_component_path}")

credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="545de186-b8b5-4f23-aba9-b7f0a043666d",
    resource_group_name="ASDP",
    workspace_name="asdp-project",
)

# Memuat komponen dari file YAML
print(f"Membaca komponen dari {input_cosmoss_component_path}...")
input_cosmos_component = load_component(source=input_cosmoss_component_path)

# Mendaftarkan komponen ke Azure ML
print("Mendaftarkan komponen ke Azure ML Workspace...")
registered_component = ml_client.components.create_or_update(input_cosmos_component)

print(f"Komponen {registered_component.name} versi {registered_component.version} berhasil didaftarkan.")


Membaca komponen dari ./Components/4. Integrate to Cosmoss DB/CSV_to_cosmos_DB.yml...
Mendaftarkan komponen ke Azure ML Workspace...


Uploading 4. Integrate to Cosmoss DB (0.01 MBs): 100%|██████████| 8346/8346 [00:00<00:00, 718050.12it/s]




Komponen csv_to_cosmos versi 1.6 berhasil didaftarkan.


In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="545de186-b8b5-4f23-aba9-b7f0a043666d",
    resource_group_name="ASDP",
    workspace_name="asdp-project",
)

# Periksa environment yang tersedia
envs = ml_client.environments.list()
for env in envs:
    print(f"Name: {env.name}, Version: {env.version}")


Name: tensorflow-custom, Version: None
Name: promptflow_985267a3ed57197e5fdfa6f698be5ad8, Version: None
Name: aml-pytorch-mlflow, Version: None
Name: promptflow_3ea58ee89be5037799ffa670144039cf, Version: None
Name: promptflow_b5ca15dd7eccef2407b40ed8a7c0fe21, Version: None
Name: AzureML-ACPT-pytorch-1.13-py38-cuda11.7-gpu, Version: None


## Training Model - Semua Data Disimpan jadi Output

In [4]:
from azure.ai.ml import MLClient, load_component
from azure.identity import DefaultAzureCredential
import os

# Path ke file predict.yml
train_svm_component_path = "./Components/2. Training dan Predict Model/oc_one_class_svm_model_all_data.yml"

# Pastikan file YAML ada
if not os.path.exists(train_svm_component_path):
    raise FileNotFoundError(f"File oc_lstm_training_model.yml tidak ditemukan di {train_svm_component_path}")

# Memuat komponen dari file YAML
print(f"Membaca komponen dari {train_svm_component_path}...")
train_svm_component = load_component(source=train_svm_component_path)

# Mendaftarkan komponen ke Azure ML
print("Mendaftarkan komponen ke Azure ML Workspace...")
registered_component = ml_client.components.create_or_update(train_svm_component)

print(f"Komponen {registered_component.name} versi {registered_component.version} berhasil didaftarkan.")

Membaca komponen dari ./Components/2. Training dan Predict Model/oc_one_class_svm_model_all_data.yml...
Mendaftarkan komponen ke Azure ML Workspace...


Uploading 2. Training dan Predict Model (0.03 MBs): 100%|██████████| 28570/28570 [00:00<00:00, 618187.22it/s]




Komponen oc_svm_model_training_v3 versi 1.8 berhasil didaftarkan.
